In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from utils.dataloader import bin_avg, retrieve_data, dataloader, \
      drop_settlement_dup, load_data, frame_to_numpy, create_rolling_window_dataset
import os
from cycler import cycler

In [5]:
# there was a bug in the r-squared metric computation, so we have to correct it now

def calculate_r_oos_mask_test(y_true, y_pred,  all_points=False):
    mask = tf.cast(y_true > 0, tf.double)
    
    numerator = tf.reduce_sum(y_true * mask)  
    denominator = tf.reduce_sum(mask )      
    mean_IV = numerator / denominator  

    if not all_points:
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred) * mask)
        ss_tot = tf.reduce_sum(tf.square(y_true - mean_IV) * mask)
        r2 = 1 - ss_res / ss_tot
    else:
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred) * mask, axis=[1, 2])
        ss_tot = tf.reduce_sum(tf.square(y_true - mean_IV) * mask, axis=[1, 2])
        r2 = 1 - ss_res / ss_tot

    return r2.numpy()

In [30]:
# Get convlstm results
folder_path = "results/validation_short_ttm/ivrmse/"  # change this to your folder path
target_substring = "_convLSTMNEW"  # the substring to look for

hvalues = ['_h_1', '_h_5', '_h_10' ]
options = ['call', 'put']
wsvalues = [ 'ws_5', 'ws_21', 'ws_63']
smooths = ['smooth_True', 'smooth_False']

def print_results(h_step, option, smooth, ws):
    lowest_value = float('inf')
    lowest_file_short = None
    i = 0
    for file_name in os.listdir(folder_path):
    
        if file_name.endswith('.npy') and smooth in file_name and ws in file_name and target_substring in file_name and h_step in file_name and option in file_name:
            i=i+1
            file_path = os.path.join(folder_path, file_name)
            try:
                value = np.load(file_path)
                # If the file contains a single number
            
                if value < lowest_value:
                    lowest_value = value
                    lowest_file_short = file_name
                    

            except Exception as e:
                print(f"Error loading {file_name}: {e}")

    # print(f"Lowest value: {lowest_value}")
    # print(f"From file: {lowest_file_short}")
    # print(i)

    _, _, smooth, _, ws_short, _, h_short, num_layers, k_height, k_width, _ = lowest_file_short.split("_")
    test_ivrmse = np.load(f"results/test_short_ttm/ivrmse/{lowest_file_short}")
    test_roos = np.load(f"results/test_short_ttm/r_oos/{lowest_file_short}")
    test_surface = np.load(f"results/test_short_ttm/surface/{lowest_file_short}")
    test_surface_pred = np.load(f"results/test_short_ttm/surface_pred/{lowest_file_short}")
    covtransformer_ivrmse_h = np.load(f"results/test_short_ttm/ivrmse_h/{lowest_file_short}")

    print(f"h = {h_step} ahead prediction for {option} option with window_size {ws} and {smooth}:")
    print(f"IVRMSE: {test_ivrmse:.3f}, R_oos: {test_roos:.3f}")
    print(f"Best hyperparameters: window size {ws}, number of layers: {num_layers}, kernel height: {k_height}, kernel width: {k_width}")

    print(f"Corrected R-squared: {calculate_r_oos_mask_test(test_surface, test_surface_pred):.3f}")
for smooth in smooths:
     for ws in wsvalues: 
        for option in options:
            for h_step in hvalues:
                print_results(h_step, option, smooth, ws)

h = _h_1 ahead prediction for call option with window_size ws_5 and True:
IVRMSE: 0.171, R_oos: 0.943
Best hyperparameters: window size ws_5, number of layers: 3, kernel height: 7, kernel width: 4
Corrected R-squared: 0.529
h = _h_5 ahead prediction for call option with window_size ws_5 and True:
IVRMSE: 0.196, R_oos: 0.926
Best hyperparameters: window size ws_5, number of layers: 3, kernel height: 3, kernel width: 4
Corrected R-squared: 0.375
h = _h_10 ahead prediction for call option with window_size ws_5 and True:
IVRMSE: 0.184, R_oos: 0.934
Best hyperparameters: window size ws_5, number of layers: 3, kernel height: 7, kernel width: 4
Corrected R-squared: 0.434
h = _h_1 ahead prediction for put option with window_size ws_5 and True:
IVRMSE: 0.229, R_oos: 0.878
Best hyperparameters: window size ws_5, number of layers: 2, kernel height: 3, kernel width: 4
Corrected R-squared: 0.214
h = _h_5 ahead prediction for put option with window_size ws_5 and True:
IVRMSE: 0.284, R_oos: 0.813
Bes